In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate gradio

In [ ]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from google.colab import drive, userdata
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import gradio as gr
import torch
import json

In [3]:
# Constants

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [5]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [27]:
def generate_instruction_response(topic, instructions, responses, num_inputs):

    instructions = instructions.strip().split("\n")
    responses = responses.strip().split("\n")

    if len(instructions) != len(responses):
        raise ValueError("Number of instructions and responses must be the same.")

    system_message = f"""
    You are a helpful assistant. Your task is to generate dataset.
    Topic: {topic}
    Return the dataset in JSON format.
    Do not repeat the provided examples.
    Return {num_inputs + len(responses)} of examples in the dataset.
    """

    messages = [{"role": "system", "content": system_message}]

    for instruction, response in zip(instructions, responses):
        messages.append({"role": "user", "content": instruction})
        messages.append({"role": "assistant", "content": response})

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

    streamer = TextStreamer(tokenizer)
    outputs = model.generate(inputs, max_new_tokens=5000, streamer=streamer)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [28]:
def create_gradio_interface():
    with gr.Blocks() as demo:
        with gr.Column():
            gr.Markdown("### Instruction-Response Dataset Generator")
            gr.Markdown("Define a topic and provide instruction-response pairs to generate a dataset.")

            topic_input = gr.Textbox(label="Topic", placeholder="Enter the topic for your dataset", lines=1)

            instructions_input = gr.Textbox(label="Instructions", placeholder="Enter instructions (separate multiple by new lines)", lines=5)

            responses_input = gr.Textbox(label="Responses", placeholder="Enter responses (separate multiple by new lines)", lines=5)

            num_inputs = gr.Number(label="Number of examples to generate", value=10)

            generate_button = gr.Button("Generate Dataset")

            outputs=gr.Textbox(label="Generated Dataset")

            generate_button.click(generate_instruction_response,
                                  inputs=[topic_input, instructions_input, responses_input, num_inputs],
                                  outputs=outputs)

    return demo

In [ ]:
demo = create_gradio_interface()
demo.launch()